In [1]:
import pandas as pd

df2 = pd.read_sql_query("SELECT * FROM gvs.vw_ml_land_only_glm_ready_v2;", engine)
df2.shape, df2.columns.tolist()[:20]


NameError: name 'engine' is not defined

In [2]:
from dotenv import load_dotenv
from pathlib import Path
import os

from sqlalchemy import create_engine
from urllib.parse import quote_plus

# шлях до .env
env_path = Path(r"C:\OneDrive\gvs-kyiv\env\.env")
load_dotenv(env_path)

host = os.getenv("PGHOST", "localhost")
port = int(os.getenv("PGPORT", "5432"))
user = os.getenv("PGUSER")
pwd  = os.getenv("PGPASSWORD")
db   = os.getenv("PGDATABASE")

assert all([user, pwd, db]), "Missing DB credentials"

url = f"postgresql+psycopg2://{user}:{quote_plus(pwd)}@{host}:{port}/{db}"
engine = create_engine(url, pool_pre_ping=True)

# smoke test
with engine.connect() as c:
    print(c.exec_driver_sql(
        "select current_database(), current_user"
    ).fetchone())


('gvs_kyiv', 'postgres')


In [3]:
import pandas as pd

df2 = pd.read_sql_query(
    "SELECT * FROM gvs.vw_ml_land_only_glm_ready_v2;",
    engine
)

df2.shape, df2.columns.tolist()[:30]


((52, 30),
 ['spatial_unit_id',
  'y_ln_uah_m2',
  'x_ln_area',
  'lon',
  'lat',
  'x_ln_dist_metro',
  'x_ln_dist_school',
  'x_ln_dist_kindergarten',
  'x_ln_dist_park',
  'x_ln_dist_water',
  'x_ln_dist_cemetery',
  'x_ln_dist_highway',
  'x_ln_dist_powerline',
  'gp_band_1',
  'gp_band_2',
  'gp_band_3',
  'x_ln_osm_edu_school',
  'x_ln_osm_edu_kindergarten',
  'x_ln_osm_health_hospital',
  'x_ln_osm_health_pharmacy',
  'x_ln_osm_retail_supermarket',
  'x_ln_osm_subway_entrance',
  'x_ln_osm_railway',
  'x_ln_osm_leisure_park',
  'x_ln_cnt500_school',
  'x_ln_cnt500_kindergarten',
  'x_ln_cnt500_hospital',
  'x_ln_cnt500_pharmacy',
  'x_ln_cnt500_pt_stops',
  'x_ln_cnt500_parks'])

In [4]:
df2.shape, df2.columns.tolist()[:30]

((52, 30),
 ['spatial_unit_id',
  'y_ln_uah_m2',
  'x_ln_area',
  'lon',
  'lat',
  'x_ln_dist_metro',
  'x_ln_dist_school',
  'x_ln_dist_kindergarten',
  'x_ln_dist_park',
  'x_ln_dist_water',
  'x_ln_dist_cemetery',
  'x_ln_dist_highway',
  'x_ln_dist_powerline',
  'gp_band_1',
  'gp_band_2',
  'gp_band_3',
  'x_ln_osm_edu_school',
  'x_ln_osm_edu_kindergarten',
  'x_ln_osm_health_hospital',
  'x_ln_osm_health_pharmacy',
  'x_ln_osm_retail_supermarket',
  'x_ln_osm_subway_entrance',
  'x_ln_osm_railway',
  'x_ln_osm_leisure_park',
  'x_ln_cnt500_school',
  'x_ln_cnt500_kindergarten',
  'x_ln_cnt500_hospital',
  'x_ln_cnt500_pharmacy',
  'x_ln_cnt500_pt_stops',
  'x_ln_cnt500_parks'])

In [5]:
pd.read_sql_query("""
select column_name
from information_schema.columns
where table_schema='gvs' and table_name='vw_ml_land_only_glm_ready_v2'
order by ordinal_position;
""", engine)


,column_name
0,spatial_unit_id
1,y_ln_uah_m2
2,lon
3,lat
4,x_ln_area
5,x_ln_dist_metro
6,x_ln_dist_school
7,x_ln_dist_kindergarten
8,x_ln_dist_park
9,x_ln_dist_water


In [6]:
df2 = pd.read_sql_query(
    "SELECT * FROM gvs.vw_ml_land_only_glm_ready_v2;",
    engine
)

df2.shape, df2.columns.tolist()[:30]

((144, 25),
 ['spatial_unit_id',
  'y_ln_uah_m2',
  'lon',
  'lat',
  'x_ln_area',
  'x_ln_dist_metro',
  'x_ln_dist_school',
  'x_ln_dist_kindergarten',
  'x_ln_dist_park',
  'x_ln_dist_water',
  'x_ln_dist_cemetery',
  'x_ln_dist_highway',
  'x_ln_dist_powerline',
  'gp_band_1',
  'gp_band_2',
  'gp_band_3',
  'x_ln_osm_edu_school',
  'x_ln_osm_edu_kindergarten',
  'x_ln_osm_health_hospital',
  'x_ln_osm_health_pharmacy',
  'x_ln_osm_retail_supermarket',
  'x_ln_osm_pt_stops',
  'x_ln_osm_subway_entrance',
  'x_ln_osm_railway',
  'x_ln_osm_leisure_park'])

In [7]:
base_cols = [
    "x_ln_area",
    "lon", "lat",
    "x_ln_dist_metro",
    "x_ln_dist_school",
    "x_ln_dist_kindergarten",
    "x_ln_dist_park",
    "x_ln_dist_water",
    "x_ln_dist_cemetery",
    "x_ln_dist_highway",
    "x_ln_dist_powerline",
    "gp_band_1", "gp_band_2", "gp_band_3",
]


In [8]:
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

y = df2["y_ln_uah_m2"].astype(float)
X_base = df2[base_cols].astype(float)
X_base = sm.add_constant(X_base)

m_base = sm.OLS(y, X_base).fit(cov_type="HC3")
print(m_base.summary())


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.856
Method:                 Least Squares   F-statistic:                     65.51
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           1.47e-51
Time:                        22:44:18   Log-Likelihood:                -170.77
No. Observations:                 144   AIC:                             371.5
Df Residuals:                     129   BIC:                             416.1
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   -138

In [9]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
y_true, y_pred = [], []

for tr, te in loo.split(X_base):
    m = sm.OLS(y.iloc[tr], X_base.iloc[tr]).fit()
    y_hat = m.predict(X_base.iloc[te])
    y_true.append(y.iloc[te].values[0])
    y_pred.append(y_hat.values[0])

y_true = np.array(y_true)
y_pred = np.array(y_pred)

r2_log_base = r2_score(y_true, y_pred)
mae_base = mean_absolute_error(np.exp(y_true), np.exp(y_pred))
mape_base = np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

r2_log_base, mae_base, mape_base


(0.8384527067926836, 474154.4796053285, np.float64(84.32413440275171))

In [10]:
osm_cols = [c for c in df2.columns if c.startswith("x_ln_osm_")]
cnt_cols = [c for c in df2.columns if c.startswith("x_ln_cnt500_")]

full_cols = base_cols + osm_cols + cnt_cols


In [11]:
X_full = df2[full_cols].astype(float)
X_full = sm.add_constant(X_full)

m_full = sm.OLS(y, X_full).fit(cov_type="HC3")
print(m_full.summary())


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     68.26
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           2.69e-56
Time:                        22:45:07   Log-Likelihood:                -165.48
No. Observations:                 144   AIC:                             371.0
Df Residuals:                     124   BIC:                             430.4
Df Model:                          19                                         
Covariance Type:                  HC3                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

C:\Users\User\anaconda3\envs\gvs\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 23, but rank is 19
  warnings.warn('covariance of constraints does not have full '


In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, r2_score

y = df2["y_ln_uah_m2"].astype(float)

# фічі-кандидати
base_cols = ["x_ln_area","lon","lat","gp_band_1","gp_band_2","gp_band_3",
             "x_ln_dist_metro","x_ln_dist_school","x_ln_dist_kindergarten","x_ln_dist_park",
             "x_ln_dist_water","x_ln_dist_cemetery","x_ln_dist_highway","x_ln_dist_powerline"]

osm_cols = [c for c in df2.columns if c.startswith("x_ln_osm_")]
cnt_cols = [c for c in df2.columns if c.startswith("x_ln_cnt500_")]


In [13]:
X_base = df2[base_cols].astype(float)
X_base = sm.add_constant(X_base, has_constant="add")
m_base = sm.OLS(y, X_base).fit(cov_type="HC3")
print(m_base.summary())


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.856
Method:                 Least Squares   F-statistic:                     65.51
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           1.47e-51
Time:                        22:54:52   Log-Likelihood:                -170.77
No. Observations:                 144   AIC:                             371.5
Df Residuals:                     129   BIC:                             416.1
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   -138

In [14]:
# приклад "OSM замість dist"
base_cols_no_overlap = [c for c in base_cols if c not in [
    "x_ln_dist_school","x_ln_dist_kindergarten","x_ln_dist_metro","x_ln_dist_park"
]]

X_osm = df2[base_cols_no_overlap + osm_cols + cnt_cols].astype(float)

# прибрати константні колонки (часта причина сингулярності)
const_like = [c for c in X_osm.columns if X_osm[c].nunique(dropna=True) <= 1]
X_osm = X_osm.drop(columns=const_like)

X_osm = sm.add_constant(X_osm, has_constant="add")
m_osm = sm.OLS(y, X_osm).fit(cov_type="HC3")
print("dropped const:", const_like)
print(m_osm.summary())


dropped const: []
                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     68.26
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           2.69e-56
Time:                        22:55:14   Log-Likelihood:                -165.48
No. Observations:                 144   AIC:                             371.0
Df Residuals:                     124   BIC:                             430.4
Df Model:                          19                                         
Covariance Type:                  HC3                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------


In [15]:
full_cols = [c for c in df2.columns if c not in ["spatial_unit_id","y_ln_uah_m2"]]
X_full = df2[full_cols].astype(float)

const_like = [c for c in X_full.columns if X_full[c].nunique(dropna=True) <= 1]
X_full = X_full.drop(columns=const_like)

X_full = sm.add_constant(X_full, has_constant="add")
m_full = sm.OLS(y, X_full).fit(cov_type="HC3")
print("dropped const:", const_like)
print(m_full.summary())


dropped const: []
                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     68.26
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           2.69e-56
Time:                        22:55:49   Log-Likelihood:                -165.48
No. Observations:                 144   AIC:                             371.0
Df Residuals:                     124   BIC:                             430.4
Df Model:                          19                                         
Covariance Type:                  HC3                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------


C:\Users\User\anaconda3\envs\gvs\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 23, but rank is 19
  warnings.warn('covariance of constraints does not have full '


In [16]:
def metrics_from_model(m, X, y_log):
    y_pred_log = m.predict(X)

    r2_log = r2_score(y_log, y_pred_log)

    y_true = np.exp(y_log)
    y_pred = np.exp(y_pred_log)

    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return r2_log, mae, mape

print("BASE:", metrics_from_model(m_base, X_base, y))
print("OSM:", metrics_from_model(m_osm, X_osm, y))
print("FULL:", metrics_from_model(m_full, X_full, y))


BASE: (0.8699157180331125, 432341.318268561, np.float64(72.90541117708258))
OSM: (0.8791346031615207, 372276.15744878736, np.float64(68.9554923226409))
FULL: (0.8791346031615205, 372276.15744874935, np.float64(68.95549232269835))


In [17]:
from sklearn.model_selection import train_test_split

bins = pd.qcut(y, q=5, duplicates="drop")
Xtr, Xte, ytr, yte = train_test_split(X_base, y, test_size=0.2, random_state=42, stratify=bins)

m_tr = sm.OLS(ytr, Xtr).fit(cov_type="HC3")
yte_pred = m_tr.predict(Xte)

print("R2 log test:", r2_score(yte, yte_pred))
print("MAE грн/м2 test:", mean_absolute_error(np.exp(yte), np.exp(yte_pred)))
print("MAPE % test:", np.mean(np.abs((np.exp(yte)-np.exp(yte_pred))/np.exp(yte))) * 100)



R2 log test: 0.8556489831446734
MAE грн/м2 test: 318555.1777545214
MAPE % test: 73.63609076158797
